In [8]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ufmgu07q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ufmgu07q
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
pip install open_clip_torch

In [13]:
from PIL import Image
import requests
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32') #16
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')

In [14]:
import os
image_folder = '/content/drive/MyDrive/Unique_BACH_folder'  # Specifica il percorso alla cartella delle immagini
image_files = [f for f in os.listdir(image_folder) if f.endswith(('tif'))]

In [15]:
import torch
from PIL import Image

In [16]:
import numpy as np

y_pred = []

# Carica il modello e le trasformazioni
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:wisdomik/QuiltNet-B-32')

# Prepara il testo
#corispondenza classe t0, t1, t2, t3

texts = ["Breast normal breast tissue","Breast non-malignant benign tissue","Breast malignant in-situ carcinoma", "Breast malignant invasive carcinoma",]
tokenizer = open_clip.get_tokenizer('hf-hub:wisdomik/QuiltNet-B-32')

# Itera su ogni immagine nel folder
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)

    # Preprocessa l'immagine usando le trasformazioni di validazione
    image_preprocessed = preprocess_val(image).unsqueeze(0)  # aggiungi una dimensione batch

    # Tokenizza il testo e calcola la lunghezza massima dei token per ogni batch di testi
    tokenized_texts = [tokenizer.encode(text) for text in texts]
    max_len = max(len(tokens) for tokens in tokenized_texts)

    # Padding dinamico dei testi per ogni batch
    padded_texts = [torch.nn.functional.pad(torch.tensor(tokens), (0, max_len - len(tokens)), 'constant', 0) for tokens in tokenized_texts]
    text_inputs = torch.stack(padded_texts)

    # Sposta il modello e gli input su GPU se disponibile
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    image_preprocessed = image_preprocessed.to(device)
    text_inputs = text_inputs.to(device)

    # Passa gli input preprocessati al modello
    with torch.no_grad():
        image_features = model.encode_image(image_preprocessed)

        # Truncate or pad the text inputs to match the expected length of positional embeddings
        expected_length = model.positional_embedding.shape[0] # Get the expected length from the model
        text_inputs = torch.nn.functional.pad(text_inputs, (0, expected_length - text_inputs.shape[1]), 'constant', 0)  # Pad if shorter
        text_inputs = text_inputs[:, :expected_length]  # Truncate if longer

        text_features = model.encode_text(text_inputs)

    # Calcola le similarità tra immagine e testi
    logits_per_image = (image_features @ text_features.T).softmax(dim=-1)

    max_value, max_index = torch.max(logits_per_image, dim=1)
    posizione = "" #valore di supporto per fare cose


    if (max_index.item() == 3):
      posizione = "Invasive"
    elif (max_index.item() == 2):
      posizione = "InSitu"
    elif (max_index.item() == 1):
      posizione = "Benign"
    else:
      posizione = "Normal"


    #Aggiungere gli elementi ad una lista con soli nomi + labels predette
    y_pred.append((image_file, posizione))






    #Stampa i risultati
    #print(f"Logits per immagine {image_file}:" )
    #print(f"Il valore massimo è {max_value.item()} e si trova in posizione {max_index.item()}")
    #print("\n")

In [38]:
#Ora stampo il mio vettore di labels prodotteeeeeeeeeeeeee
print("Valori PREDETTI ----------------------------------------------------------------")

for coppia in y_pred:
  if coppia[0] ==  "n001.tif":
    print("Ci sono")
  else:
    print(coppia)
    print("\n")

print("La lunghezza della lista PRED è:", len(y_pred)) #400 IL MISTERO SI INFITTISCE




Valori PREDETTI ----------------------------------------------------------------
('is013.tif', 'Invasive')


('is007.tif', 'Invasive')


('is004.tif', 'Invasive')


('is010.tif', 'Invasive')


('is011.tif', 'Invasive')


('is005.tif', 'Invasive')


('is039.tif', 'InSitu')


('is015.tif', 'Invasive')


('is014.tif', 'Invasive')


('is016.tif', 'Invasive')


('is002.tif', 'Invasive')


('is058.tif', 'Invasive')


('is064.tif', 'Invasive')


('is065.tif', 'Invasive')


('is071.tif', 'Invasive')


('is067.tif', 'Invasive')


('is099.tif', 'Invasive')


('is066.tif', 'Invasive')


('is062.tif', 'InSitu')


('is063.tif', 'Invasive')


('is075.tif', 'InSitu')


('is061.tif', 'InSitu')


('is049.tif', 'Invasive')


('is074.tif', 'Invasive')


('is079.tif', 'Invasive')


('is100.tif', 'InSitu')


('is051.tif', 'Invasive')


('is087.tif', 'Invasive')


('is050.tif', 'Invasive')


('is046.tif', 'Invasive')


('is052.tif', 'Invasive')


('is085.tif', 'Invasive')


('is091.tif', 'Invasive')


('is0

CODICE CHE LAVORA SULLE LABELS VERE:

In [43]:
import pandas as pd

# Leggi il file Excel
df = pd.read_csv('/content/drive/MyDrive/WE/microscopy_ground_truth.csv')


# Inizializza la lista per le coppie
y_vere = []


#-----------------------------------------------------DOVREBBE ESSERCI IL PROBLEMA
# Itera sul DataFrame e crea le coppie

for index, row in df.iterrows():
    coppia = (row[0], row[1])  # Leggi la colonna 0 e la colonna 1 e crea la coppia
    y_vere.append(coppia)  # Aggiungi la coppia alla lista

# Stampa la lista di coppie


print("Valori reali ----------------------------------------------------------------")

for coppia in y_vere:
      print(coppia)
      print("\n")




print("La lunghezza della lista VERE è:", len(y_vere)) #400


#CODCIE FIXATOOOOOOOOOOOO

Valori reali ----------------------------------------------------------------
C'e anche il 001-------------------------------
('n002.tif', 'Normal')


('n003.tif', 'Normal')


('n004.tif', 'Normal')


('n005.tif', 'Normal')


('n006.tif', 'Normal')


('n007.tif', 'Normal')


('n008.tif', 'Normal')


('n009.tif', 'Normal')


('n010.tif', 'Normal')


('n011.tif', 'Normal')


('n012.tif', 'Normal')


('n013.tif', 'Normal')


('n014.tif', 'Normal')


('n015.tif', 'Normal')


('n016.tif', 'Normal')


('n017.tif', 'Normal')


('n018.tif', 'Normal')


('n019.tif', 'Normal')


('n020.tif', 'Normal')


('n021.tif', 'Normal')


('n022.tif', 'Normal')


('n023.tif', 'Normal')


('n024.tif', 'Normal')


('n025.tif', 'Normal')


('n026.tif', 'Normal')


('n027.tif', 'Normal')


('n028.tif', 'Normal')


('n029.tif', 'Normal')


('n030.tif', 'Normal')


('n031.tif', 'Normal')


('n032.tif', 'Normal')


('n033.tif', 'Normal')


('n034.tif', 'Normal')


('n035.tif', 'Normal')


('n036.tif', 'Normal')



Sezione del codice in cui, dopo aver verificato la corretta creazione delle due liste andiamo a calcolare l'accuracyyyyyyyy

In [25]:
#NON COMPILARE PIUUUUUUUUUUUUU SOLO PER BACKUP DI CODICE

#Andiamo a ripulire il contenuto di y_vere
y_vere = y_vere[:-2]
#verifico la composizione della lista a cui ho inserito l'elemento. Consiglio di fare l'inserimento in una cella separata in maniera tale da aggiungerlo solo quando necessario<
for coppia in y_vere:
    print(coppia)
    print("\n")
#Da qui in poi non modificheremo piu il contenuto di y_vere

('n002.tif', 'Normal')


('n003.tif', 'Normal')


('n004.tif', 'Normal')


('n005.tif', 'Normal')


('n006.tif', 'Normal')


('n007.tif', 'Normal')


('n008.tif', 'Normal')


('n009.tif', 'Normal')


('n010.tif', 'Normal')


('n011.tif', 'Normal')


('n012.tif', 'Normal')


('n013.tif', 'Normal')


('n014.tif', 'Normal')


('n015.tif', 'Normal')


('n016.tif', 'Normal')


('n017.tif', 'Normal')


('n018.tif', 'Normal')


('n019.tif', 'Normal')


('n020.tif', 'Normal')


('n021.tif', 'Normal')


('n022.tif', 'Normal')


('n023.tif', 'Normal')


('n024.tif', 'Normal')


('n025.tif', 'Normal')


('n026.tif', 'Normal')


('n027.tif', 'Normal')


('n028.tif', 'Normal')


('n029.tif', 'Normal')


('n030.tif', 'Normal')


('n031.tif', 'Normal')


('n032.tif', 'Normal')


('n033.tif', 'Normal')


('n034.tif', 'Normal')


('n035.tif', 'Normal')


('n036.tif', 'Normal')


('n037.tif', 'Normal')


('n038.tif', 'Normal')


('n039.tif', 'Normal')


('n040.tif', 'Normal')


('n041.tif', 'Normal')




In [44]:
#una volta ottenute le due liste, voglio andare ad ordinarmi la lista valori reali

#confornto prima di tutto se le due liste contengono lo stesso numero di elementi
if len(y_pred) == len(y_vere):
    print("Le due liste hanno la stessa lunghezza.")
else:
    print("Le due liste hanno lunghezze diverse.")


#andiamo a debuggare -- - vabbe------------------:)---mandiamo avanti il codice RICORDA DI FIXAREEE
print("La lunghezza della lista PRED è:", len(y_pred))
print("La lunghezza della lista VERE è:", len(y_vere))



# Inizializzazione della lista di coppie ordinata
ORDINATA = []


#STACOSANONFUNZIONA
my_counter = 0


Le due liste hanno la stessa lunghezza.
La lunghezza della lista PRED è: 400
La lunghezza della lista VERE è: 400


In [45]:
# Converti le liste in set per poter fare l'operazione di differenza
set_b = set(y_vere)
set_a = set(y_pred)

# Trova le coppie che sono in B ma non in A
differenza = set_b - set_a

# Converti il risultato in una lista (se necessario)
differenza_lista = list(differenza)

# Stampa la differenza
print("Coppie presenti in B ma non in A:", differenza_lista)


Coppie presenti in B ma non in A: [('n061.tif', 'Normal'), ('is078.tif', 'InSitu'), ('iv068.tif', 'Invasive'), ('is022.tif', 'InSitu'), ('is024.tif', 'InSitu'), ('b025.tif', 'Benign'), ('iv077.tif', 'Invasive'), ('n007.tif', 'Normal'), ('b019.tif', 'Benign'), ('b024.tif', 'Benign'), ('n095.tif', 'Normal'), ('n013.tif', 'Normal'), ('iv098.tif', 'Invasive'), ('n088.tif', 'Normal'), ('is091.tif', 'InSitu'), ('is050.tif', 'InSitu'), ('n096.tif', 'Normal'), ('b083.tif', 'Benign'), ('n008.tif', 'Normal'), ('b068.tif', 'Benign'), ('b012.tif', 'Benign'), ('iv053.tif', 'Invasive'), ('n078.tif', 'Normal'), ('is093.tif', 'InSitu'), ('b007.tif', 'Benign'), ('b049.tif', 'Benign'), ('is042.tif', 'InSitu'), ('b053.tif', 'Benign'), ('n063.tif', 'Normal'), ('n032.tif', 'Normal'), ('b099.tif', 'Benign'), ('b046.tif', 'Benign'), ('n074.tif', 'Normal'), ('is053.tif', 'InSitu'), ('n098.tif', 'Normal'), ('n003.tif', 'Normal'), ('n028.tif', 'Normal'), ('n070.tif', 'Normal'), ('n085.tif', 'Normal'), ('iv002.t

In [46]:
for coppia in y_vere: #Il ciclo viene operato sulla stringa piu corta ed e`per questo che le due stringhe vengono di lunghezza uguale pur non essendolo.!!!!!!!-------------
    ticerco = coppia[0]  # Estrai il primo elemento della coppia corrente di A e salvalo in ticerco
    #print("\n", my_counter) #segnala l'avvenuta aggiunta di un elemento alla lista ordinata
    # Secondo ciclo for per cercare ticerco in tutti i primi elementi delle coppie presenti nella lista di coppie B
    for coppia_b in y_pred:
        # Se trovi ticerco in uno dei primi elementi delle coppie di B
        if ticerco == coppia_b[0]:
            # Aggiungi la coppia di B trovata nella lista ORDINATA
            ORDINATA.append(coppia_b)
            my_counter = my_counter + 1

print("LA LISTA PREDETTA ORDINATA e'", )

for coppia in ORDINATA:
    print(coppia)
    print("\n")



LA LISTA PREDETTA ORDINATA e'
('n001.tif', 'Invasive')


('n002.tif', 'Invasive')


('n003.tif', 'Invasive')


('n004.tif', 'Invasive')


('n005.tif', 'InSitu')


('n006.tif', 'Invasive')


('n007.tif', 'InSitu')


('n008.tif', 'Invasive')


('n009.tif', 'Invasive')


('n010.tif', 'Invasive')


('n011.tif', 'Invasive')


('n012.tif', 'Invasive')


('n013.tif', 'Invasive')


('n014.tif', 'Invasive')


('n015.tif', 'Invasive')


('n016.tif', 'Invasive')


('n017.tif', 'Invasive')


('n018.tif', 'Invasive')


('n019.tif', 'Invasive')


('n020.tif', 'Invasive')


('n021.tif', 'Invasive')


('n022.tif', 'Invasive')


('n023.tif', 'Invasive')


('n024.tif', 'Invasive')


('n025.tif', 'Invasive')


('n026.tif', 'Invasive')


('n027.tif', 'Invasive')


('n028.tif', 'Invasive')


('n029.tif', 'Invasive')


('n030.tif', 'Invasive')


('n031.tif', 'Invasive')


('n032.tif', 'Invasive')


('n033.tif', 'Invasive')


('n034.tif', 'Invasive')


('n035.tif', 'Invasive')


('n036.tif', 'Invasive')


('

In [47]:
#Quale è l'elemento che non ha un simile?

print("La lunghezza della lista PRED è:", len(ORDINATA)) #399
print("La lunghezza della lista VERE è:", len(y_vere)) #400
#stampa le lunghezze delle due stringhe per
i = 0
index = 0

# fai un for della lunghezza della lista piu lunga, in questo caso predicted labels

for coppia in ORDINATA:
    # Estrai il primo elemento dalla coppia corrente in l1
    elemento_l1 = coppia[0]
    i = index
    for pippo in y_vere:
        # Estrai il primo elemento dalla prima coppia in l2
        elemento_l2 = pippo[0] #perchè pippoi mi da solo lettere????????!
        #print(elemento_l2) #-------------------------------------------
        # Confronta i primi elementi
        if elemento_l1 == elemento_l2:
            print("Match")
            index = index+1
    if(i == index): #singifica che non è avvenuto l'aggiornamento
      print("Nessuna corrispondenza per: \n", elemento_l1)


La lunghezza della lista PRED è: 400
La lunghezza della lista VERE è: 400
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Match
Ma

In [49]:
#Per sicurezza stampati la lista A e B in un file di testo per compararle agevolmente
# Definizione della lista di coppie


# Nome del file di output
nome_file_p = '/content/drive/MyDrive/WE/lista_coppie_pred.txt'
nome_file_v = '/content/drive/MyDrive/WE/lista_coppie_vere.txt'

def scrivi_lista_in_file(lista_coppie, nome_file):
    try:
        with open(nome_file, 'w') as file:
            for coppia in lista_coppie:
                file.write(f"{coppia[0]}, {coppia[1]}\n")
        print(f"Lista di coppie scritta in {nome_file}")
    except Exception as e:
        print(f"Errore durante la scrittura nel file: {e}")

scrivi_lista_in_file(ORDINATA, nome_file_p)

scrivi_lista_in_file(y_vere, nome_file_v)



Lista di coppie scritta in /content/drive/MyDrive/WE/lista_coppie_pred.txt
Lista di coppie scritta in /content/drive/MyDrive/WE/lista_coppie_vere.txt


In [50]:
#Ora posso estrarre le labels della prima e della seconda e calcolare l'accuracy

#estraggo le labels reali
reallabels = []
predictedlabels = []

# Iterazione sulla lista di coppie
for coppia in y_vere:
    # Aggiungi il secondo elemento della coppia alla lista reallabels
    reallabels.append(coppia[1])

# Stampare la lista reallabels
print(reallabels)

#estraggo le labels predette----------------------------------------------------------
# Iterazione sulla lista di coppie
for coppia in ORDINATA:
    # Aggiungi il secondo elemento della coppia alla lista reallabels
    predictedlabels.append(coppia[1])

# Stampare la lista reallabels
print(predictedlabels)


['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal',

In [51]:
len(reallabels), len(predictedlabels)

(400, 400)

In [52]:
from sklearn.metrics import accuracy_score

#look at the parameters and their meaning
accuracy_score(reallabels, predictedlabels)

0.19